 # 課題5: 評判分析

本課題ではAmazonに投稿された映画のレビュー(英語)を分析し、レビューがPositiveかNegativeかの判別を行います。

今回の課題のディレクトリには `Training_data` と `Test_data` が入っています。さらにその中には `pos` と `neg` というディレクトリがあり、Positiveなデータが `pos` に、Negativeなデータが `neg` に入っています。

- Training_data (positive用)、文章数 : 700
- Training_data (negative用)、文章数 : 700
- Test_data (positive用)、文章数 : 3
- Test_data (negative用)、文章数 : 3

（ ※学習用データ：1400、　テスト用データ：6、　合計 : 1406 の文章です）

`Training_data`を用いて機械学習を行い、その結果を元に、6つのTest dataがPositiveかNegativeかを判別してください。

レッスン9で学んだ内容を踏まえ、各セルに'#コメント'の内容を実行するコードを記入してください。

わからない場合は、ここまでのレッスン内容や各種ライブラリの公式ドキュメントを参照しましょう。

## 1. 必要なモジュールのインポート

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 文章ファイル検索用
import glob
import collections
from sklearn.feature_extraction import DictVectorizer

# JupyterNotebook上でグラフを表示する設定
%matplotlib inline
# DataFrameで全ての列を表示する設定
pd.options.display.max_columns = None

## 2. データの読み込み

`./Amazon_review` 配下のフォルダからファイルを読み込みます。

In [2]:
# Training_dataフォルダ
train_neg_files = glob.glob("./Amazon_review/Training_data/neg/cv*")
train_pos_files = glob.glob("./Amazon_review/Training_data/pos/cv*")
# Test_dataフォルダ
test_neg_files = glob.glob("./Amazon_review/Test_data/neg/amazon*")
test_pos_files = glob.glob("./Amazon_review/Test_data/pos/amazon*")

In [3]:
# それぞれのファイル数を確認
print(len(train_neg_files))
print(len(train_pos_files))
print(len(test_neg_files))
print(len(test_pos_files))

700
700
3
3


In [4]:
# ファイル名をまとめたリスト filenames を用意
filenames = train_neg_files + train_pos_files + test_neg_files + test_pos_files
# filenamesの長さを確認
print(len(filenames))

1406


In [5]:
# エンコーディング用定数(以下で繰り返し指定するため定数にしておく)
ENCODING = 'utf-8'

In [6]:
# 最初のファイルの内容を確認
with open(filenames[0], 'r', encoding=ENCODING) as f:
    text = f.read()
    print(text)

some talented actresses are blessed with a demonstrated wide acting range while others , almost as gifted , have more limited types of parts for which they are suitable . as was amply evident after basic instinct , sharon stone can play sensual roles with great abandon . rejecting her natural abilities , she has spent the rest of her entire career trying with little success to play against type . gloria is her latest disaster . babe ruth didn't quit baseball after one season to play football in a quixotic quest to prove his athletic dexterity , and neither should stone reject what she does best . janeane garofalo , for example , is no less wonderful an actress because she could have never pulled off stone's part in basic instinct ; neither is stone any less talented because she couldn't do garofalo's comedic roles . gloria , directed by respected director sidney lumet and adapted by steve antin from the 1980 screenplay by john cassavetes , was not screened in advance for critics , almo

In [7]:
# Training_dataのposの最初のファイルの内容を確認
with open(train_pos_files[0], 'r', encoding=ENCODING) as f:
    text = f.read()
    print(text)

filmcritic . com presents a review from staff member bradley null you can find the review with full credits at <a href= " http : //filmcritic . com/misc/emporium . nsf/2a460f93626cd4678625624c007f2b46/4273eccb750e44ab882569680006d5fc ? opendocument " >http : //filmcritic . com/misc/emporium . nsf/2a460f93626cd4678625624c007f2b46/4273eccb750e44ab882569680006d5fc ? opendocument</a> here's the pitch : take an emotional drama about the racial conflict concerning the integration of a black high school and a white one in the south . then wrap the entire plot around a hard-nosed high school football coach ( washington ) with an unorthodox style but an uncanny ability to get the most out of his players . as an added little twist in this case , the old white head coach ( patton ) stays on as an assistant so we can play with a fair amount of racial conflict and power struggle as these two egos collide , and ultimately generate a little more emotion as they become friends . the last bit aside , w

## 3. データの前処理

In [8]:
# 文字列の中で使われている単語ごとの数を返す
def get_word_count(text, min_length=3):
    # ノイズの除去：不要と思われる文字を除去する
    for ch in ".,:;!?-+*/=()[]{}<>~^#$@%&'\"_0123456789":
        text = text.replace(ch, ' ')

    # 形態素解析：文章を単語に分割
    _words = text.strip().split()

    # 表記のゆれの補正：
    # 単語のリストを受け取り、指定された文字数以上の単語だけを全て小文字にして返す
    _words = [_word.lower() for _word in _words if len(_word) >= min_length]

    # collections.Counterの戻り値は辞書型のサブクラス
    _count = collections.Counter(_words)

    # 辞書型に変換して返す
    return dict(_count)

In [9]:
# 最初のファイルでテスト
with open(filenames[0], 'r', encoding=ENCODING) as f:
    text = f.read()
    
get_word_count(text)

{'some': 1,
 'talented': 2,
 'actresses': 1,
 'are': 3,
 'blessed': 1,
 'with': 7,
 'demonstrated': 1,
 'wide': 1,
 'acting': 1,
 'range': 1,
 'while': 1,
 'others': 1,
 'almost': 3,
 'gifted': 1,
 'have': 4,
 'more': 2,
 'limited': 1,
 'types': 1,
 'parts': 1,
 'for': 9,
 'which': 1,
 'they': 3,
 'suitable': 1,
 'was': 2,
 'amply': 1,
 'evident': 1,
 'after': 6,
 'basic': 2,
 'instinct': 2,
 'sharon': 1,
 'stone': 6,
 'can': 3,
 'play': 3,
 'sensual': 1,
 'roles': 2,
 'great': 1,
 'abandon': 1,
 'rejecting': 1,
 'her': 5,
 'natural': 1,
 'abilities': 1,
 'she': 9,
 'has': 3,
 'spent': 1,
 'the': 27,
 'rest': 2,
 'entire': 1,
 'career': 1,
 'trying': 2,
 'little': 1,
 'success': 1,
 'against': 1,
 'type': 1,
 'gloria': 7,
 'latest': 1,
 'disaster': 1,
 'babe': 1,
 'ruth': 1,
 'didn': 1,
 'quit': 1,
 'baseball': 1,
 'one': 2,
 'season': 1,
 'football': 1,
 'quixotic': 1,
 'quest': 1,
 'prove': 1,
 'his': 4,
 'athletic': 1,
 'dexterity': 1,
 'and': 16,
 'neither': 2,
 'should': 1,
 'reje

In [10]:
# 単語ごとの数のリスト word_count_data を作成
word_count_data = []

for filename in filenames:
    with open(filename, 'r', encoding=ENCODING) as f:
        text = f.read()
        count = get_word_count(text)
        word_count_data.append(count)

In [11]:
# 単語ごとの数のリストの長さを確認
print(len(word_count_data))

1406


In [12]:
# リストの0番目を表示
word_count_data[0]

{'some': 1,
 'talented': 2,
 'actresses': 1,
 'are': 3,
 'blessed': 1,
 'with': 7,
 'demonstrated': 1,
 'wide': 1,
 'acting': 1,
 'range': 1,
 'while': 1,
 'others': 1,
 'almost': 3,
 'gifted': 1,
 'have': 4,
 'more': 2,
 'limited': 1,
 'types': 1,
 'parts': 1,
 'for': 9,
 'which': 1,
 'they': 3,
 'suitable': 1,
 'was': 2,
 'amply': 1,
 'evident': 1,
 'after': 6,
 'basic': 2,
 'instinct': 2,
 'sharon': 1,
 'stone': 6,
 'can': 3,
 'play': 3,
 'sensual': 1,
 'roles': 2,
 'great': 1,
 'abandon': 1,
 'rejecting': 1,
 'her': 5,
 'natural': 1,
 'abilities': 1,
 'she': 9,
 'has': 3,
 'spent': 1,
 'the': 27,
 'rest': 2,
 'entire': 1,
 'career': 1,
 'trying': 2,
 'little': 1,
 'success': 1,
 'against': 1,
 'type': 1,
 'gloria': 7,
 'latest': 1,
 'disaster': 1,
 'babe': 1,
 'ruth': 1,
 'didn': 1,
 'quit': 1,
 'baseball': 1,
 'one': 2,
 'season': 1,
 'football': 1,
 'quixotic': 1,
 'quest': 1,
 'prove': 1,
 'his': 4,
 'athletic': 1,
 'dexterity': 1,
 'and': 16,
 'neither': 2,
 'should': 1,
 'reje

### 行列への変換

In [13]:
# DictVectorizerを使用して行列に変換し、datasetに格納する
vec = DictVectorizer()
dataset = vec.fit_transform(word_count_data)

In [14]:
# datasetの大きさを確認
dataset

<1406x35075 sparse matrix of type '<class 'numpy.float64'>'
	with 445276 stored elements in Compressed Sparse Row format>

In [15]:
# 各列に対応した単語を取得
vec.get_feature_names()

['``the',
 '`accidentally',
 '`apollo',
 '`armageddon',
 '`at',
 '`atmosphere',
 '`batman',
 '`bats',
 '`believers',
 '`ben',
 '`bio',
 '`blade',
 '`blue',
 '`bonnie',
 '`boob',
 '`chef',
 '`city',
 '`closure',
 '`cooks',
 '`cops',
 '`cruel',
 '`cute',
 '`dad',
 '`dammnit',
 '`damn',
 '`dark',
 '`darlene',
 '`dawson',
 '`daylight',
 '`diddly',
 '`disclaiming',
 '`down',
 '`dragonheart',
 '`drive',
 '`dumb',
 '`d駛a',
 '`eat',
 '`elvis',
 '`empire',
 '`enough',
 '`ev',
 '`ex',
 '`experience',
 '`father',
 '`fatty',
 '`fiji',
 '`frank',
 '`friends',
 '`gangsta',
 '`georgia',
 '`give',
 '`go',
 '`hang',
 '`hanging',
 '`hello',
 '`homage',
 '`horror',
 '`hot',
 '`how',
 '`if',
 '`inspector',
 '`it',
 '`jaws',
 '`jedi',
 '`jeez',
 '`just',
 '`kids',
 '`kirk',
 '`kyle',
 '`last',
 '`life',
 '`like',
 '`long',
 '`loose',
 '`macho',
 '`marketing',
 '`mission',
 '`not',
 '`oh',
 '`one',
 '`padawan',
 '`parental',
 '`political',
 '`power',
 '`praise',
 '`pump',
 '`ready',
 '`return',
 '`ronna',
 

## 4. 機械学習の実施

In [16]:
# 必要なライブラリのインポート
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [17]:
# 目的変数 Y の用意
# neg700 + pos700 + neg3 + pos3 = 1406
Y = [0]*700 + [1]*700 + [0]*3 + [1]*3

In [28]:
# データの分割
# dataset    の先頭から1400件を 変数 X_train に、残りの6件を 変数 X_test に代入
# 目的変数 Y の先頭から1400件を 変数 Y_train に、残りの6件を Y_test に代入
X_train = dataset[:1400]
X_test = dataset[1400:]
Y_train = Y[:1400]
Y_test = Y[1400:]
# X_trainとY_trainを、train_test_splitを使って7:3に分割
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.3, random_state=0)

In [29]:
# ロジスティック回帰モデルの作成
logistic_model = LogisticRegression(max_iter=1000)
# モデルの学習
logistic_model.fit(X_train, Y_train)
# 検証データによる予測
Y_pred = logistic_model.predict(X_valid)

# 検証データによるモデルの評価
print(classification_report(Y_valid, Y_pred))

              precision    recall  f1-score   support

           0       0.76      0.78      0.77       207
           1       0.78      0.76      0.77       213

    accuracy                           0.77       420
   macro avg       0.77      0.77      0.77       420
weighted avg       0.77      0.77      0.77       420



## 5. テストデータによる評価

最後にテストデータによる評価を行いましょう

In [30]:
# テストデータによる予測
Y_pred = logistic_model.predict(X_test)

# モデルの評価
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.67      0.67      0.67         3
           1       0.67      0.67      0.67         3

    accuracy                           0.67         6
   macro avg       0.67      0.67      0.67         6
weighted avg       0.67      0.67      0.67         6

